# Project in Machine Learning
Classification of objects - Pictures of Cats, Dogs and Wild Animals

In this project, we use a training set of 14630 pictures (512x512, jpg) of cats, dogs and wild animals in order to create a model that can recognize the animals in new images. We, then, test this model in a validation set of 1500 pictures.

Source of the dataset: https://www.kaggle.com/datasets/andrewmvd/animal-faces/ \
Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition \
Yunjey Choi and Youngjung Uh and Jaejun Yoo and Jung-Woo Ha, 2020 \
StarGAN v2: Diverse Image Synthesis for Multiple Domains